In [3]:
import pandas as pd
import numpy as np
import IPython.display as ipd
import os
import librosa
import csv

In [16]:
ls data/UrbanSound8K/metadata/

UrbanSound8K.csv


In [47]:
from src.helpers import data_to_df

In [49]:
ls data/UrbanSound8K/audio/fold1/7061-6-0-0.wav

data/UrbanSound8K/audio/fold1/7061-6-0-0.wav


In [13]:
metadata = pd.read_csv('data/UrbanSound8K/metadata/UrbanSound8K.csv')

In [54]:
ls data/UrbanSound8K/audio/fold1/101415-3-0-2.wav

fold1/  fold10/ fold2/  fold3/  fold4/  fold5/  fold6/  fold7/  fold8/  fold9/


In [50]:
metadata.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [21]:
ipd.Audio('data/UrbanSound8K/audio/fold1/180937-7-3-26.wav')

### maybe we can make a feature like variable energy?
1. take the energy for timestamps along the interval
2. plot as wave
3. get frequency and amplitude of wave?

In [124]:
def make_feature_file():
    header = 'YTID chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
    for i in range(1, 21):
        header += f' mfcc{i}'
    header = header.split()
    file = open('data/c3_features.csv', 'w', newline='')
    with file:
        writer = csv.writer(file)
        writer.writerow(header)
make_feature_file()

In [125]:
ls data

UrbanSound8K/                class_labels_indices.csv
audioset_v1_embeddings/      df_reduced.csv
balanced_train_segments.csv  extracted_features.csv
c3_features.csv              qa_true_counts.csv
c3_features.csv.csv


In [59]:
ls data/UrbanSound8K/audio/fold1/101415-3-0-2.wav

fold1/  fold10/ fold2/  fold3/  fold4/  fold5/  fold6/  fold7/  fold8/  fold9/


In [178]:
def featurize(wav_file, y, sr):
    filename = wav_file
    rmse = librosa.feature.rms(y=y)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
    for e in mfcc:
        to_append += f' {np.mean(e)}'
        
    write_to_file(to_append)
        
def get_features(base_path):
    i = 0
    folds = []
    for i in range(1, 11):
        folds.append('fold' + str(i) + '/')
    
    for fold in folds:
        fold_path = base_path + fold
        path_list = os.listdir(fold_path)
#         path_list = os.listdir(os.path.join(base_path, fold))

        for wav_file in path_list:
            if wav_file != '.DS_Store':
                filename = fold_path + wav_file

                y, sr = librosa.load(filename, res_type='kaiser_fast')
                feat = featurize(filename, y, sr)
                i += 1
                if i%500 == 0:
                    print(i)
            
def write_to_file(row):            
    file = open('data/c3_initial_features.csv', 'a', newline='')
    with file:
        writer = csv.writer(file)
        writer.writerow(row.split())
    

In [179]:
get_features('data/UrbanSound8K/audio/')

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500


## Read in data to pd dataframe

In [4]:
df = pd.read_csv('data/c3_initial_features.csv')
header = 'YTID chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header = header.split()
df = pd.read_csv('data/c3_initial_features.csv', names=header)
df.head()

,YTID,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,data/UrbanSound8K/audio/fold1/135776-2-0-49.wav,0.352791,0.008648,2118.703260,1812.417148,4016.315791,0.131181,-366.268707,110.406792,-56.866329,...,-11.154856,8.294682,-10.794812,-2.265402,-12.632551,-1.060647,-7.977907,-2.894937,-3.895293,5.808671
1,data/UrbanSound8K/audio/fold1/46654-6-0-0.wav,0.602263,0.127402,2713.997357,1532.644414,4339.371094,0.208369,-110.229149,91.258995,-137.438950,...,-14.399264,2.538663,-2.090154,2.588182,2.170496,2.481971,0.419651,-3.368144,3.145062,-4.766719
2,data/UrbanSound8K/audio/fold1/57320-0-0-24.wav,0.571699,0.060016,2105.109929,2138.264862,4320.954604,0.094684,-204.144394,83.016212,-33.412800,...,11.281404,5.459733,1.599511,18.695854,-7.685503,18.432896,0.935115,16.634794,-3.238273,7.388971
3,data/UrbanSound8K/audio/fold1/134717-0-0-26.wav,0.493135,0.098351,867.165494,1307.835866,1334.062839,0.038425,-197.093430,186.151215,-4.336608,...,-13.271729,-3.329089,-23.252317,-15.378189,-17.651825,-5.364439,-15.198536,-2.484705,-0.467842,0.983536
4,data/UrbanSound8K/audio/fold1/174276-7-5-0.wav,0.689207,0.060016,3654.752411,2626.493157,6860.582709,0.250063,-105.484627,54.665283,-22.048914,...,-4.381224,24.335947,-9.858241,16.636402,-9.498604,9.891488,-6.868745,8.067618,-3.427906,2.707397


In [5]:
df['YTID'] = [idx.split('/')[3] + '/' + idx.split('/')[4] for idx in df['YTID']]
df.head()

,YTID,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,fold1/135776-2-0-49.wav,0.352791,0.008648,2118.703260,1812.417148,4016.315791,0.131181,-366.268707,110.406792,-56.866329,...,-11.154856,8.294682,-10.794812,-2.265402,-12.632551,-1.060647,-7.977907,-2.894937,-3.895293,5.808671
1,fold1/46654-6-0-0.wav,0.602263,0.127402,2713.997357,1532.644414,4339.371094,0.208369,-110.229149,91.258995,-137.438950,...,-14.399264,2.538663,-2.090154,2.588182,2.170496,2.481971,0.419651,-3.368144,3.145062,-4.766719
2,fold1/57320-0-0-24.wav,0.571699,0.060016,2105.109929,2138.264862,4320.954604,0.094684,-204.144394,83.016212,-33.412800,...,11.281404,5.459733,1.599511,18.695854,-7.685503,18.432896,0.935115,16.634794,-3.238273,7.388971
3,fold1/134717-0-0-26.wav,0.493135,0.098351,867.165494,1307.835866,1334.062839,0.038425,-197.093430,186.151215,-4.336608,...,-13.271729,-3.329089,-23.252317,-15.378189,-17.651825,-5.364439,-15.198536,-2.484705,-0.467842,0.983536
4,fold1/174276-7-5-0.wav,0.689207,0.060016,3654.752411,2626.493157,6860.582709,0.250063,-105.484627,54.665283,-22.048914,...,-4.381224,24.335947,-9.858241,16.636402,-9.498604,9.891488,-6.868745,8.067618,-3.427906,2.707397


In [6]:
df.head()

,YTID,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,fold1/135776-2-0-49.wav,0.352791,0.008648,2118.703260,1812.417148,4016.315791,0.131181,-366.268707,110.406792,-56.866329,...,-11.154856,8.294682,-10.794812,-2.265402,-12.632551,-1.060647,-7.977907,-2.894937,-3.895293,5.808671
1,fold1/46654-6-0-0.wav,0.602263,0.127402,2713.997357,1532.644414,4339.371094,0.208369,-110.229149,91.258995,-137.438950,...,-14.399264,2.538663,-2.090154,2.588182,2.170496,2.481971,0.419651,-3.368144,3.145062,-4.766719
2,fold1/57320-0-0-24.wav,0.571699,0.060016,2105.109929,2138.264862,4320.954604,0.094684,-204.144394,83.016212,-33.412800,...,11.281404,5.459733,1.599511,18.695854,-7.685503,18.432896,0.935115,16.634794,-3.238273,7.388971
3,fold1/134717-0-0-26.wav,0.493135,0.098351,867.165494,1307.835866,1334.062839,0.038425,-197.093430,186.151215,-4.336608,...,-13.271729,-3.329089,-23.252317,-15.378189,-17.651825,-5.364439,-15.198536,-2.484705,-0.467842,0.983536
4,fold1/174276-7-5-0.wav,0.689207,0.060016,3654.752411,2626.493157,6860.582709,0.250063,-105.484627,54.665283,-22.048914,...,-4.381224,24.335947,-9.858241,16.636402,-9.498604,9.891488,-6.868745,8.067618,-3.427906,2.707397


In [7]:
df.to_csv('data/c3_final_features.csv', sep='\t', index=False)

In [8]:
## Build a nn

In [9]:
from keras import layers
from keras.models import Sequential
import warnings

In [10]:
## let's see how the metadata is set up

In [14]:
ls data/UrbanSound8K/metadata/

UrbanSound8K.csv


In [19]:
metadata = pd.read_csv('data/UrbanSound8K/metadata/UrbanSound8K.csv')
metadata.head(20)

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.000000,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.500000,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.500000,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.000000,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.500000,72.500000,1,5,2,children_playing
5,100263-2-0-143.wav,100263,71.500000,75.500000,1,5,2,children_playing
6,100263-2-0-161.wav,100263,80.500000,84.500000,1,5,2,children_playing
7,100263-2-0-3.wav,100263,1.500000,5.500000,1,5,2,children_playing
8,100263-2-0-36.wav,100263,18.000000,22.000000,1,5,2,children_playing
9,100648-1-0-0.wav,100648,4.823402,5.471927,2,10,1,car_horn


In [20]:
metadata[metadata['class'] == 'children_playing']

,slice_file_name,fsID,start,end,salience,fold,classID,class
1,100263-2-0-117.wav,100263,58.5,62.5,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.5,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.0,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.5,1,5,2,children_playing
5,100263-2-0-143.wav,100263,71.5,75.5,1,5,2,children_playing
...,...,...,...,...,...,...,...,...
8712,99500-2-0-23.wav,99500,11.5,15.5,2,9,2,children_playing
8713,99500-2-0-29.wav,99500,14.5,18.5,2,9,2,children_playing
8714,99500-2-0-39.wav,99500,19.5,23.5,2,9,2,children_playing
8715,99500-2-0-41.wav,99500,20.5,24.5,2,9,2,children_playing


In [23]:
metadata['fsID'].nunique()

1297